In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import time
import decimal

### THIS NOTEBOOK LOADS THE PMPM DATASET GETS THE NECESSARY AMOUNT OF COLUMNS AND SAVES LOCALY THE CREATED DATASET AS WELL AS in S3

### Workaround for the problem of loading the whole data: first get the columns and then populate them 1 by 1

In [3]:
from pathlib import Path
Path('../../temp/').mkdir(exist_ok = 'True')

import boto3
s3 = boto3.client('s3')
bucket = 'cdphp-s3-us-e-p-pond'

if not Path('../../temp/pre_female_tr').exists():
    s3.download_file(bucket, 'rpi/xshou/female_tr', '../../temp/pre_female_tr')
    

for file in Path('../../temp/').iterdir():
    if file.is_file():
        if 'pre' in file.name:
            print(file.name)
            df =  pd.read_pickle(file)
    
columns = list(df.columns)
del df

pre_female_tr


In [4]:
columns_upper = [c.upper() for c in columns]

### Read PMPM with pyarrow (seems to be problematic, we will use spark)


path = '../data/PMPM/'
pmpm = pq.read_table(path)

### Read PMPM with pyspark

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , column

# initialise sparkContext
spark = SparkSession.builder \
    .master('local') \
    .appName('myAppName') \
    .config('spark.executor.memory', '20gb') \
    .config("spark.cores.max", "1") \
    .getOrCreate()

sc = spark.sparkContext


from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


path = '../../data/PMPM/'
pmpm=sqlContext.read.parquet(path)


### Get the Columns of PMPM names

In [6]:
columns = pmpm.columns
columns[-1]

'IS_CANARY_ELIGIBLE'

### Filter By Eligibility

In [7]:
pmpm_eligible = pmpm.filter(pmpm.IS_CANARY_ELIGIBLE == 1)

### Regular Expressions to Filter out Unessesary columns

In [8]:
import re
schema = pmpm_eligible.schema.names
match = re.compile(r'^PERS_ID|^CC[\w\s]|^ALLOW_AMT[\w\s]*|MYR|AGE_AT_MIDMONTH|MBR_GNDR')

columns_to_keep = []
columns_to_drop = []
for sh in schema:
    if match.match(sh) is not None:
        columns_to_keep.append(sh)
    else:
        columns_to_drop.append(sh)


### Get the columns to keep on the final eligibility array

In [9]:
total_cols = np.intersect1d(columns_to_keep, columns)
len(total_cols)

73

In [10]:
dr = len(columns_to_drop)
kp = len(columns_to_keep)
total = len(schema)
dr, kp, dr+kp, total

(655, 76, 731, 731)

### Select only Columns of Interest

In [11]:
pmpm_eligible2 = pmpm_eligible.select(columns_to_keep)

In [12]:
del pmpm, pmpm_eligible

In [13]:
from pyspark.sql.functions import col , column
from pyspark.sql.types import *
pmpm_eligible2 = pmpm_eligible2.select(
    [col(name) if ('decimal' not in colType) or( name=='MYR') or (name == 'PERS_ID') else col(name).cast(IntegerType()) for name, colType in pmpm_eligible2.dtypes ]
)

In [14]:
pmpm_eligible2.dtypes

[('PERS_ID', 'string'),
 ('MYR', 'decimal(38,10)'),
 ('MBR_GNDR', 'string'),
 ('AGE_AT_MIDMONTH', 'int'),
 ('ALLOW_AMT', 'int'),
 ('ALLOW_AMT_PAID', 'int'),
 ('ALLOW_AMT_CAP', 'int'),
 ('CC_HYPOTHYROID', 'int'),
 ('CC_AMI', 'int'),
 ('CC_ALZHEIMER', 'int'),
 ('CC_ALZHEIMER_PLUS', 'int'),
 ('CC_ANEMIA', 'int'),
 ('CC_ASTHMA', 'int'),
 ('CC_ATRIAL_FIB', 'int'),
 ('CC_BPH', 'int'),
 ('CC_CATARACT', 'int'),
 ('CC_COPD', 'int'),
 ('CC_DEPRESSION', 'int'),
 ('CC_GLAUCOMA', 'int'),
 ('CC_HIP_FRACTURE', 'int'),
 ('CC_HYPERLIPIDEMIA', 'int'),
 ('CC_HYPERTENSION', 'int'),
 ('CC_OSTEOPOROSIS', 'int'),
 ('CC_STROKE', 'int'),
 ('CC_BREAST_CANCER', 'int'),
 ('CC_COLORECTAL_CANCER', 'int'),
 ('CC_PROSTATE_CANCER', 'int'),
 ('CC_LUNG_CANCER', 'int'),
 ('CC_ENDOMETRIAL_CANCER', 'int'),
 ('CC_CHRONIC_KIDNEY', 'int'),
 ('CC_DIABETES', 'int'),
 ('CC_HEART_FAILURE', 'int'),
 ('CC_ISCHEMIC', 'int'),
 ('CC_ARTHRITIS', 'int'),
 ('CC_AUTISM', 'int'),
 ('CC_CEREBRAL_PALSY', 'int'),
 ('CC_CYSTIC_FIBROSIS', 'int'

### Get only the necessary columns from pmpm dataset 

In [15]:
st = time.time()
df = pmpm_eligible2.select('PERS_ID').toPandas()
en = time.time() - st
print(en)
st1 = time.time()
for i, coll in enumerate(total_cols):
    if i%10 == 0:
        print(i)
        print(coll)
    st = time.time()
    df[coll] = pmpm_eligible2.select(coll).toPandas()
    print(time.time() - st)
print(time.time() - st1)

195.41280341148376
0
AGE_AT_MIDMONTH
99.47015476226807
141.31494116783142
96.3345582485199
101.02346277236938
95.23339509963989
114.21460485458374
94.85086607933044
110.57521033287048
104.17756271362305
108.0151903629303
10
CC_ATRIAL_FIB
101.66873955726624
111.96190214157104
107.7175600528717
110.58435702323914
113.2296679019928
125.37529230117798
102.07022857666016
105.62785339355469
101.61114835739136
113.13349485397339
20
CC_CYSTIC_FIBROSIS
108.57443594932556
109.35505604743958
98.66494250297546
126.05697059631348
109.31452059745789
117.20204186439514
114.25304365158081
101.8342604637146
102.27179408073425
125.91637301445007
30
CC_GLAUCOMA
98.87104105949402
114.44795489311218
105.89500856399536
105.90996861457825
110.7874231338501
100.055992603302
106.57513332366943
116.10252380371094
108.01505374908447
98.83730244636536
40
CC_HEPATITIS_E
115.97447299957275
99.18540716171265
115.5703592300415
111.25331616401672
106.76975321769714
115.50017952919006
106.85034322738647
115.84507465362

In [21]:
5+5

10

#type(eligible.AGE_AT_MIDMONTH[0]) == decimal.Decimal

for column in df.columns:
    typi = type(df[column][0])
    if typi == decimal.Decimal:
        df[column] = df[column].astype(int)

### decimal.Decimal class takes up to 16 bytes per number leading to an explosion of memory, so we make these columns to Integers

#type(eligible.AGE_AT_MIDMONTH[0]) == decimal.Decimal

for column in eligible.columns:
    typi = type(eligible[column][0])
    if typi == decimal.Decimal:
        eligible[column] = eligible[column].astype(int)

In [16]:
#eligible.memory_usage().sum()/10**9
df.memory_usage().sum()/10**9

2.476585624

### Save Dataset in pickle format

In [27]:
#eligible.to_pickle('../temp/eligible.p')
df.MYR = df.MYR.astype(int)
df.to_pickle('../../temp/eligible.p')

In [18]:
df.head()

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2570122               95      40392        0             0   
1  2570122               94      19837        0             0   
2  2570268               93       4922        0             1   
3  2570268               94        232        0             1   
4  2570268               93       2776        0             1   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  1       0          1                    0             0   
1                  0       0          1                    0             1   
2                  1       0          1                    0             1   
3                  1       0          1                    0             1   
4                  1       0          1                    0             1   

   ...  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  \
0  ...                 0                       1               0   
1  ...                 0                       0               0   
2  ...                 0                       0               0   
3  ...                 0                       0               0   
4  ...                 0                       0               0   

   CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  \
0                0          0           0                   0   
1                0          0           0                   0   
2                0          0           0                   0   
3                0          0           0                   0   
4                0          0           0                   0   

   CC_VISUAL_IMPAIR  MBR_GNDR                MYR  
0                 0         M  202002.0000000000  
1                 0         M  201810.0000000000  
2                 0         F  201704.0000000000  
3                 0         F  201807.0000000000  
4                 0         F  201801.0000000000  

[5 rows x 74 columns]

### Upload Dataset in S3 for Fastest Download

In [28]:
# Save dataset to S3
s3.upload_file('../../temp/eligible.p', bucket, 'rpi/DATA_GEORGE/PMPM_PICKLE/ELIGIBLE.p', ExtraArgs = {'ACL':"bucket-owner-full-control", 'ServerSideEncryption':'AES256'})